In [1]:
from pyspark import SparkContext
sc = SparkContext(appName="MY-APP-NAME", master="local[*]")

from pyspark.sql import SQLContext
sqlCtx = SQLContext(sc)

from pyspark.sql.functions import *
import pandas as pd
from pyspark.mllib.stat import Statistics

In [2]:
df = sqlCtx.read.load("database_Homicide.csv",
                     format="csv", sep=",", inferSchema="true", header="true")

In [3]:
# DataFrame
pandas_df = df.select("*").toPandas()
pandas_df

,Record ID,Agency Code,Agency Name,Agency Type,City,State,Year,Month,Incident,Crime Type,...,Victim Ethnicity,Perpetrator Sex,Perpetrator Age,Perpetrator Race,Perpetrator Ethnicity,Relationship,Weapon,Victim Count,Perpetrator Count,Record Source
0,1,AK00101,Anchorage,Municipal Police,Anchorage,Alaska,1980,January,1,Murder or Manslaughter,...,Unknown,Male,15,Native American/Alaska Native,Unknown,Acquaintance,Blunt Object,0,0,FBI
1,2,AK00101,Anchorage,Municipal Police,Anchorage,Alaska,1980,March,1,Murder or Manslaughter,...,Unknown,Male,42,White,Unknown,Acquaintance,Strangulation,0,0,FBI
2,3,AK00101,Anchorage,Municipal Police,Anchorage,Alaska,1980,March,2,Murder or Manslaughter,...,Unknown,Unknown,0,Unknown,Unknown,Unknown,Unknown,0,0,FBI
3,4,AK00101,Anchorage,Municipal Police,Anchorage,Alaska,1980,April,1,Murder or Manslaughter,...,Unknown,Male,42,White,Unknown,Acquaintance,Strangulation,0,0,FBI
4,5,AK00101,Anchorage,Municipal Police,Anchorage,Alaska,1980,April,2,Murder or Manslaughter,...,Unknown,Unknown,0,Unknown,Unknown,Unknown,Unknown,0,1,FBI
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
638449,638450,WY01500,Park County,Sheriff,Park,Wyoming,2014,January,1,Murder or Manslaughter,...,Hispanic,Unknown,0,Unknown,Unknown,Unknown,Handgun,0,0,FBI
638450,638451,WY01700,Sheridan County,Sheriff,Sheridan,Wyoming,2014,June,1,Murder or Manslaughter,...,Unknown,Male,57,White,Unknown,Acquaintance,Handgun,0,0,FBI
638451,638452,WY01701,Sheridan,Municipal Police,Sheridan,Wyoming,2014,September,1,Murder or Manslaughter,...,Unknown,Female,22,Asian/Pacific Islander,Unknown,Daughter,Suffocation,0,0,FBI
638452,638453,WY01800,Sublette County,Sheriff,Sublette,Wyoming,2014,December,1,Murder or Manslaughter,...,Not Hispanic,Male,31,White,Not Hispanic,Stranger,Knife,0,1,FBI


In [4]:
print("The number of rows and columns: ",df.count(),",",len(df.columns))

The number of rows and columns:  638454 , 24


In [5]:
# Columns type
df.dtypes

[('Record ID', 'int'),
 ('Agency Code', 'string'),
 ('Agency Name', 'string'),
 ('Agency Type', 'string'),
 ('City', 'string'),
 ('State', 'string'),
 ('Year', 'int'),
 ('Month', 'string'),
 ('Incident', 'int'),
 ('Crime Type', 'string'),
 ('Crime Solved', 'string'),
 ('Victim Sex', 'string'),
 ('Victim Age', 'int'),
 ('Victim Race', 'string'),
 ('Victim Ethnicity', 'string'),
 ('Perpetrator Sex', 'string'),
 ('Perpetrator Age', 'string'),
 ('Perpetrator Race', 'string'),
 ('Perpetrator Ethnicity', 'string'),
 ('Relationship', 'string'),
 ('Weapon', 'string'),
 ('Victim Count', 'int'),
 ('Perpetrator Count', 'int'),
 ('Record Source', 'string')]

In [6]:
# Count distinct values columns
for col in df.columns:
    x=df.select(col).distinct().count()
    print('Column',col, 'has',x,'distinct values')

Column Record ID has 638454 distinct values
Column Agency Code has 12003 distinct values
Column Agency Name has 9216 distinct values
Column Agency Type has 7 distinct values
Column City has 1782 distinct values
Column State has 51 distinct values
Column Year has 35 distinct values
Column Month has 12 distinct values
Column Incident has 1000 distinct values
Column Crime Type has 2 distinct values
Column Crime Solved has 2 distinct values
Column Victim Sex has 3 distinct values
Column Victim Age has 101 distinct values
Column Victim Race has 5 distinct values
Column Victim Ethnicity has 3 distinct values
Column Perpetrator Sex has 3 distinct values
Column Perpetrator Age has 101 distinct values
Column Perpetrator Race has 5 distinct values
Column Perpetrator Ethnicity has 3 distinct values
Column Relationship has 28 distinct values
Column Weapon has 16 distinct values
Column Victim Count has 11 distinct values
Column Perpetrator Count has 11 distinct values
Column Record Source has 2 dis

In [7]:
# First drop
df=df.drop('Record ID','Incident','Victim Count','Perpetrator Count')

In [8]:
# Renamed column
df = df.withColumnRenamed("Agency Code", "Agency_Code")\
 .withColumnRenamed("Agency Name", "Agency_Name")\
.withColumnRenamed("Agency Type", "Agency_Type")\
.withColumnRenamed("Crime Type", "Crime_Type")\
.withColumnRenamed("Crime Solved", "Crime_Solved")\
.withColumnRenamed("Victim Age", "Victim_Age")\
.withColumnRenamed("Victim Sex", "Victim_Sex")\
.withColumnRenamed("Victim Race", "Victim_Race")\
.withColumnRenamed("Victim Ethnicity", "Victim_Ethnicity")\
.withColumnRenamed("Perpetrator Sex", "Perpetrator_Sex")\
.withColumnRenamed("Perpetrator Age", "Perpetrator_Age")\
.withColumnRenamed("Perpetrator Race", "Perpetrator_Race")\
.withColumnRenamed("Perpetrator Ethnicity", "Perpetrator_Ethnicity")\
.withColumnRenamed("Record Source", "Record_Source")

In [9]:
# Create view
df.createOrReplaceTempView("homicide")

In [10]:
print("The number of rows and columns: ",df.count(),",",len(df.columns))

The number of rows and columns:  638454 , 20


## Correlazione

In [11]:
# Year
X= sqlCtx.sql("SELECT Year FROM homicide")
# Trasformo data frame in RDD per usarlo nella funzione corr
Xseries= X.rdd.map(lambda p: p.Year)

# Victim_Age
Y= sqlCtx.sql("SELECT Victim_Age FROM homicide")
Yseries= Y.rdd.map(lambda p: p.Victim_Age)

print("Correlation (Year, Victim_Age): " + str(Statistics.corr(Xseries, Yseries, method="pearson")))

Correlation (Year, Victim_Age): 0.005513650636452138


## Agency Type

In [12]:
atype = sqlCtx.sql("SELECT Agency_Type FROM homicide")

In [13]:
atype.distinct().show()

+----------------+
|     Agency_Type|
+----------------+
|         Sheriff|
|  Special Police|
| Regional Police|
|Municipal Police|
|    State Police|
|   County Police|
|   Tribal Police|
+----------------+



In [14]:
# Count distinct values
atype_count = sqlCtx.sql("SELECT Agency_Type, count(*) as Count FROM homicide GROUP BY Agency_Type")

In [15]:
atype_count.show()

+----------------+------+
|     Agency_Type| Count|
+----------------+------+
|         Sheriff|105322|
|  Special Police|  2889|
| Regional Police|   235|
|Municipal Police|493026|
|    State Police| 14235|
|   County Police| 22693|
|   Tribal Police|    54|
+----------------+------+



In [16]:
# Count and Percentage 
view_targ= sqlCtx.sql("SELECT Agency_Type, count(*) as TotAgency FROM homicide GROUP BY Agency_Type")
view_targ.createOrReplaceTempView("view_targ")

In [17]:
atype_target = sqlCtx.sql("SELECT h.Agency_Type, Crime_Solved, count(*) as Count, count(*)/v.TotAgency as PercCount FROM homicide as h, view_targ as v WHERE v.Agency_Type=h.Agency_Type GROUP BY h.Agency_Type, Crime_Solved, v.TotAgency ORDER BY h.Agency_Type")

In [18]:
atype_target.show()

+----------------+------------+------+-------------------+
|     Agency_Type|Crime_Solved| Count|          PercCount|
+----------------+------------+------+-------------------+
|   County Police|          No|  7533| 0.3319525844974221|
|   County Police|         Yes| 15160| 0.6680474155025778|
|Municipal Police|         Yes|336009| 0.6815238952915262|
|Municipal Police|          No|157017| 0.3184761047084738|
| Regional Police|          No|    49|0.20851063829787234|
| Regional Police|         Yes|   186| 0.7914893617021277|
|         Sheriff|          No| 22328|0.21199749340118873|
|         Sheriff|         Yes| 82994| 0.7880025065988112|
|  Special Police|         Yes|  2058| 0.7123572170301142|
|  Special Police|          No|   831|0.28764278296988577|
|    State Police|         Yes| 11715| 0.8229715489989463|
|    State Police|          No|  2520|0.17702845100105374|
|   Tribal Police|          No|     4|0.07407407407407407|
|   Tribal Police|         Yes|    50| 0.925925925925925

## Agency Code

In [19]:
acode = sqlCtx.sql("SELECT Agency_Code FROM homicide")

In [20]:
acode.distinct().count()

12003

## Agency Name

In [21]:
aname = sqlCtx.sql("SELECT Agency_Name FROM homicide")
aname.distinct().count()

9216

In [22]:
aname.show(20)

+-----------+
|Agency_Name|
+-----------+
|  Anchorage|
|  Anchorage|
|  Anchorage|
|  Anchorage|
|  Anchorage|
|  Anchorage|
|  Anchorage|
|  Anchorage|
|  Anchorage|
|  Anchorage|
|  Anchorage|
|  Anchorage|
|  Anchorage|
|  Anchorage|
|  Anchorage|
|  Anchorage|
|  Anchorage|
|     Juneau|
|       Nome|
|     Bethel|
+-----------+
only showing top 20 rows



In [23]:
aname = sqlCtx.sql("SELECT Agency_Name, Agency_Code FROM homicide GROUP BY Agency_Name, Agency_Code ORDER BY Agency_Name")

In [24]:
# Same Agency_Name with different Agency_Code
# Agency_Code is unique
aname.show()

+-----------------+-----------+
|      Agency_Name|Agency_Code|
+-----------------+-----------+
|        Abbeville|    SC00101|
|        Abbeville|    LA05701|
|        Abbeville|    SC00100|
|        Abbeville|    AL03701|
| Abbeville County|    SC00100|
|         Aberdeen|    OH00801|
|         Aberdeen|    WA01401|
|         Aberdeen|    SD00601|
|         Aberdeen|    MD01301|
|         Aberdeen|    MS04801|
|         Aberdeen|    NC06301|
|         Aberdeen|    ID00601|
|Aberdeen Township|    NJ01330|
|        Abernathy|    TX09501|
|          Abilene|    TX22101|
|          Abilene|    KS02101|
|         Abingdon|    IL04801|
|         Abingdon|    VA09401|
|         Abington|    MA01201|
|Abington Township|    PA04601|
+-----------------+-----------+
only showing top 20 rows



## Year

In [25]:
year= sqlCtx.sql("SELECT Year FROM homicide")

In [26]:
year.describe().show()

+-------+-----------------+
|summary|             Year|
+-------+-----------------+
|  count|           638454|
|   mean|1995.801102350365|
| stddev|9.927693313985529|
|    min|             1980|
|    max|             2014|
+-------+-----------------+



In [27]:
year.distinct().count()

35

In [28]:
year_count = sqlCtx.sql("SELECT Year, count(*) as Count FROM homicide GROUP BY Year ORDER BY Year")
year_count.show(35)

+----+-----+
|Year|Count|
+----+-----+
|1980|23092|
|1981|21208|
|1982|20544|
|1983|19653|
|1984|18093|
|1985|18386|
|1986|20125|
|1987|18783|
|1988|18779|
|1989|19868|
|1990|21246|
|1991|22657|
|1992|23793|
|1993|24335|
|1994|23246|
|1995|21179|
|1996|18969|
|1997|17794|
|1998|16061|
|1999|14621|
|2000|14671|
|2001|15803|
|2002|16268|
|2003|16512|
|2004|16233|
|2005|16836|
|2006|17275|
|2007|17303|
|2008|15595|
|2009|15840|
|2010|15121|
|2011|14756|
|2012|15033|
|2013|14445|
|2014|14331|
+----+-----+



## Month

In [29]:
month= sqlCtx.sql("SELECT Month FROM homicide")

In [30]:
month_count = sqlCtx.sql("SELECT Month, count(*) as Count FROM homicide GROUP BY Month")
month_count.show()

+---------+-----+
|    Month|Count|
+---------+-----+
|     July|58696|
| November|50016|
| February|46079|
|  January|52928|
|    March|51444|
|  October|53650|
|      May|53394|
|   August|58072|
|    April|51209|
|     June|53662|
| December|55187|
|September|54117|
+---------+-----+



In [31]:
# Crime Solved distribution
view_targ= sqlCtx.sql("SELECT Month, count(*) as Tot FROM homicide GROUP BY Month")
view_targ.createOrReplaceTempView("view_targ")
target = sqlCtx.sql("SELECT h.Month, Crime_Solved, count(*) as Count, count(*)/v.Tot as PercCount FROM homicide as h, view_targ as v WHERE v.Month=h.Month GROUP BY h.Month, Crime_Solved, v.Tot ORDER BY h.Month")
target.show()

+--------+------------+-----+-------------------+
|   Month|Crime_Solved|Count|          PercCount|
+--------+------------+-----+-------------------+
|   April|         Yes|36613| 0.7149719775820657|
|   April|          No|14596|0.28502802241793435|
|  August|         Yes|40805|  0.702662212425954|
|  August|          No|17267|  0.297337787574046|
|December|          No|17958|0.32540272165546236|
|December|         Yes|37229| 0.6745972783445376|
|February|          No|12821|0.27823954512901755|
|February|         Yes|33258| 0.7217604548709824|
| January|         Yes|37951| 0.7170306831922612|
| January|          No|14977| 0.2829693168077388|
|    July|         Yes|41468|   0.70648766525828|
|    July|          No|17228|0.29351233474172006|
|    June|          No|15845|0.29527412321568336|
|    June|         Yes|37817| 0.7047258767843166|
|   March|          No|14804| 0.2877692247881191|
|   March|         Yes|36640| 0.7122307752118808|
|     May|          No|15543| 0.2911001236093943|


In [32]:
# Year and Month
year_month= sqlCtx.sql("SELECT Year, Month, count(*) as Count FROM homicide GROUP BY Year,Month ORDER BY Year, Count(*)")
year_month.show()

+----+---------+-----+
|Year|    Month|Count|
+----+---------+-----+
|1980| February| 1730|
|1980|    March| 1743|
|1980|    April| 1760|
|1980|  January| 1825|
|1980| November| 1861|
|1980|      May| 1863|
|1980|September| 1869|
|1980|  October| 1935|
|1980| December| 1965|
|1980|     June| 1991|
|1980|     July| 2179|
|1980|   August| 2371|
|1981| December| 1533|
|1981| November| 1615|
|1981|  October| 1648|
|1981|September| 1659|
|1981|     June| 1714|
|1981|    April| 1771|
|1981| February| 1777|
|1981|   August| 1804|
+----+---------+-----+
only showing top 20 rows



## City

In [33]:
city= sqlCtx.sql("SELECT City FROM homicide")

In [34]:
# City e State
# City is not unique
city_state= sqlCtx.sql("SELECT State,City, count(*) as Count FROM homicide GROUP BY City,State ORDER BY City")
city_state.show()

+--------------+---------+-----+
|         State|     City|Count|
+--------------+---------+-----+
|South Carolina|Abbeville|   59|
|     Louisiana|   Acadia|   98|
|      Virginia| Accomack|   80|
|         Idaho|      Ada|  211|
|          Iowa|    Adair|    2|
|      Oklahoma|    Adair|   56|
|      Kentucky|    Adair|   58|
|      Missouri|    Adair|   14|
|         Idaho|    Adams|    3|
|    Washington|    Adams|   32|
|  North Dakota|    Adams|    2|
|          Iowa|    Adams|    5|
|   Mississippi|    Adams|  126|
|      Colorado|    Adams|  464|
|       Indiana|    Adams|    3|
|      Nebraska|    Adams|   18|
|          Ohio|    Adams|   12|
|      Illinois|    Adams|   27|
|  Pennsylvania|    Adams|   67|
|     Wisconsin|    Adams|   15|
+--------------+---------+-----+
only showing top 20 rows



## State

In [35]:
state= sqlCtx.sql("SELECT State FROM homicide")

In [36]:
view_targ= sqlCtx.sql("SELECT State, count(*) as Tot FROM homicide GROUP BY State")
view_targ.createOrReplaceTempView("view_targ")
target = sqlCtx.sql("SELECT h.State, Crime_Solved, count(*) as Count, count(*)/v.Tot as PercCount FROM homicide as h, view_targ as v WHERE v.State=h.State GROUP BY h.State, Crime_Solved, v.Tot ORDER BY h.State")
target.show(51)

+--------------------+------------+-----+-------------------+
|               State|Crime_Solved|Count|          PercCount|
+--------------------+------------+-----+-------------------+
|             Alabama|          No| 2400| 0.2109704641350211|
|             Alabama|         Yes| 8976| 0.7890295358649789|
|              Alaska|          No|  297| 0.1836734693877551|
|              Alaska|         Yes| 1320| 0.8163265306122449|
|             Arizona|          No| 3643| 0.2830393908787196|
|             Arizona|         Yes| 9228| 0.7169606091212803|
|            Arkansas|         Yes| 5842| 0.8409385346192602|
|            Arkansas|          No| 1105| 0.1590614653807399|
|          California|          No|36369| 0.3644809236042212|
|          California|         Yes|63414| 0.6355190763957789|
|            Colorado|          No| 1329|0.20157743060822084|
|            Colorado|         Yes| 5264| 0.7984225693917791|
|         Connecticut|          No| 1637|0.33435457516339867|
|       

## Crime Type

In [37]:
c_type= sqlCtx.sql("SELECT Crime_Type FROM homicide")
c_type.distinct().show(2,False)

+--------------------------+
|Crime_Type                |
+--------------------------+
|Manslaughter by Negligence|
|Murder or Manslaughter    |
+--------------------------+



In [38]:
type_count = sqlCtx.sql("SELECT Crime_Type, count(*) as Count FROM homicide GROUP BY Crime_Type")
type_count.show()

+--------------------+------+
|          Crime_Type| Count|
+--------------------+------+
|Manslaughter by N...|  9116|
|Murder or Manslau...|629338|
+--------------------+------+



## Crime Solved

In [39]:
c_solved= sqlCtx.sql("SELECT Crime_Solved FROM homicide")
c_solved.distinct().show(2,False)

+------------+
|Crime_Solved|
+------------+
|No          |
|Yes         |
+------------+



In [40]:
solved_count = sqlCtx.sql("SELECT Crime_Solved, count(*) as Count FROM homicide GROUP BY Crime_Solved")
solved_count.show()

+------------+------+
|Crime_Solved| Count|
+------------+------+
|          No|190282|
|         Yes|448172|
+------------+------+



## Victim e Perpetrator attributes

In [41]:
for col in df.columns:
    if ("Victim" in col) | ("Perpetrator" in col):
        print(col)
        vict= sqlCtx.sql("SELECT " +col+" FROM homicide")
        v_count = sqlCtx.sql("SELECT "+col+", count(*) as Count FROM homicide GROUP BY "+col)
        v_count.show()
        view_targ= sqlCtx.sql("SELECT "+col+", count(*) as Tot FROM homicide GROUP BY "+col)
        view_targ.createOrReplaceTempView("view_targ")
        target = sqlCtx.sql("SELECT h."+col+", Crime_Solved, count(*) as Count, count(*)/v.Tot as PercCount FROM homicide as h, view_targ as v WHERE v."+col+"=h."+col+" GROUP BY h."+col+", Crime_Solved, v.Tot ORDER BY "+col)
        target.show()
        

Victim_Sex
+----------+------+
|Victim_Sex| Count|
+----------+------+
|    Female|143345|
|   Unknown|   984|
|      Male|494125|
+----------+------+

+----------+------------+------+-------------------+
|Victim_Sex|Crime_Solved| Count|          PercCount|
+----------+------------+------+-------------------+
|    Female|          No| 33076|0.23074400920855279|
|    Female|         Yes|110269| 0.7692559907914472|
|      Male|          No|156553|0.31682873766759423|
|      Male|         Yes|337572| 0.6831712623324058|
|   Unknown|          No|   653| 0.6636178861788617|
|   Unknown|         Yes|   331| 0.3363821138211382|
+----------+------------+------+-------------------+

Victim_Age
+----------+-----+
|Victim_Age|Count|
+----------+-----+
|        31|15762|
|        85|  627|
|        65| 2418|
|        53| 4788|
|        78| 1102|
|        34|14296|
|        81|  930|
|        28|18199|
|        76| 1213|
|        27|19465|
|        26|20469|
|        44| 7921|
|        12| 1239|
| 

+---------------------+------+
|Perpetrator_Ethnicity| Count|
+---------------------+------+
|         Not Hispanic|145172|
|              Unknown|446410|
|             Hispanic| 46872|
+---------------------+------+

+---------------------+------------+------+--------------------+
|Perpetrator_Ethnicity|Crime_Solved| Count|           PercCount|
+---------------------+------------+------+--------------------+
|             Hispanic|         Yes| 46852|  0.9995733060249189|
|             Hispanic|          No|    20|4.266939750810718...|
|         Not Hispanic|          No|   226|0.001556774033560...|
|         Not Hispanic|         Yes|144946|  0.9984432259664399|
|              Unknown|          No|190036| 0.42569834905132053|
|              Unknown|         Yes|256374|  0.5743016509486795|
+---------------------+------------+------+--------------------+



In [42]:
# Victim_Race and Victim_Ethnicity
v_race= sqlCtx.sql("SELECT Victim_Race, Victim_Ethnicity, count(*) FROM homicide GROUP BY Victim_Race, Victim_Ethnicity ORDER BY Victim_Race, Victim_Ethnicity")
v_race.show()

+--------------------+----------------+--------+
|         Victim_Race|Victim_Ethnicity|count(1)|
+--------------------+----------------+--------+
|Asian/Pacific Isl...|        Hispanic|      52|
|Asian/Pacific Isl...|    Not Hispanic|    5568|
|Asian/Pacific Isl...|         Unknown|    4270|
|               Black|        Hispanic|    1213|
|               Black|    Not Hispanic|  100973|
|               Black|         Unknown|  197713|
|Native American/A...|        Hispanic|      11|
|Native American/A...|    Not Hispanic|    2010|
|Native American/A...|         Unknown|    2546|
|             Unknown|        Hispanic|     177|
|             Unknown|    Not Hispanic|     170|
|             Unknown|         Unknown|    6329|
|               White|        Hispanic|   71199|
|               White|    Not Hispanic|   88778|
|               White|         Unknown|  157445|
+--------------------+----------------+--------+



In [43]:
# Perp_Race and Perp_Ethnicity
p_race= sqlCtx.sql("SELECT Perpetrator_Race, Perpetrator_Ethnicity, count(*) FROM homicide GROUP BY Perpetrator_Race, Perpetrator_Ethnicity ORDER BY Perpetrator_Race, Perpetrator_Ethnicity")
p_race.show()

+--------------------+---------------------+--------+
|    Perpetrator_Race|Perpetrator_Ethnicity|count(1)|
+--------------------+---------------------+--------+
|Asian/Pacific Isl...|             Hispanic|      48|
|Asian/Pacific Isl...|         Not Hispanic|    3316|
|Asian/Pacific Isl...|              Unknown|    2682|
|               Black|             Hispanic|     875|
|               Black|         Not Hispanic|   75704|
|               Black|              Unknown|  137937|
|Native American/A...|             Hispanic|      17|
|Native American/A...|         Not Hispanic|    1542|
|Native American/A...|              Unknown|    2043|
|             Unknown|             Hispanic|     165|
|             Unknown|         Not Hispanic|     404|
|             Unknown|              Unknown|  195478|
|               White|             Hispanic|   45767|
|               White|         Not Hispanic|   64206|
|               White|              Unknown|  108270|
+--------------------+------

In [44]:
v_age= sqlCtx.sql("SELECT Victim_Age FROM homicide")
p_age= sqlCtx.sql("SELECT Perpetrator_Age FROM homicide ORDER BY Perpetrator_Age")

In [45]:
v_age.describe().show()

+-------+-----------------+
|summary|       Victim_Age|
+-------+-----------------+
|  count|           638454|
|   mean|35.03351220291517|
| stddev| 41.6283059837085|
|    min|                0|
|    max|              998|
+-------+-----------------+



In [46]:
# Victim_Age outliers
v_out=sqlCtx.sql("SELECT Victim_Age, count(*) as count FROM homicide WHERE Victim_Age>=90 GROUP BY Victim_Age ORDER BY Victim_Age")
v_out.show()

+----------+-----+
|Victim_Age|count|
+----------+-----+
|        90|  281|
|        91|  215|
|        92|  156|
|        93|  134|
|        94|  116|
|        95|   82|
|        96|   37|
|        97|   39|
|        98|   33|
|        99| 9281|
|       998|  974|
+----------+-----+



In [47]:
# Analysis Age= 99 and Relationship
v_test=sqlCtx.sql("SELECT Victim_Age,Relationship, count(*) as count FROM homicide WHERE Victim_Age==99 GROUP BY Victim_Age, Relationship ORDER BY Victim_Age")
v_test.show()

+----------+--------------------+-----+
|Victim_Age|        Relationship|count|
+----------+--------------------+-----+
|        99|            Employee|    5|
|        99|            Employer|    3|
|        99|           Boyfriend|   39|
|        99|        Acquaintance| 1071|
|        99|              Friend|  156|
|        99|     Common-Law Wife|   25|
|        99|             Stepson|    5|
|        99|              Family|  102|
|        99|          Ex-Husband|    6|
|        99|             Husband|   52|
|        99|            Neighbor|   47|
|        99|          Stepfather|    5|
|        99|        Stepdaughter|    6|
|        99|                 Son|  244|
|        99|            Daughter|  208|
|        99|  Common-Law Husband|   15|
|        99|              Mother|   26|
|        99|             Brother|   33|
|        99|          Stepmother|    1|
|        99|Boyfriend/Girlfriend|   10|
+----------+--------------------+-----+
only showing top 20 rows



In [48]:
# Elimination of Age=998 or 99
v_age= sqlCtx.sql("SELECT Victim_Age FROM homicide WHERE Victim_Age<99")
v_age.describe().show()

+-------+------------------+
|summary|        Victim_Age|
+-------+------------------+
|  count|            628199|
|   mean| 32.59542756355868|
| stddev|16.032934863712622|
|    min|                 0|
|    max|                98|
+-------+------------------+



In [49]:
p_age.describe().show()

+-------+-----------------+
|summary|  Perpetrator_Age|
+-------+-----------------+
|  count|           638454|
|   mean|20.32269720715542|
| stddev|17.88684181199937|
|    min|                 |
|    max|               99|
+-------+-----------------+



In [50]:
p_out=sqlCtx.sql("SELECT Perpetrator_Age, Crime_Solved, count(*) as count FROM homicide WHERE Perpetrator_Age==' ' GROUP BY Perpetrator_Age, Crime_Solved ORDER BY Perpetrator_Age")
p_out.show()

+---------------+------------+-----+
|Perpetrator_Age|Crime_Solved|count|
+---------------+------------+-----+
|               |          No|    1|
+---------------+------------+-----+



In [51]:
# Missing value row
pandas_df = df.select("*").where("Perpetrator_Age==' ' ").toPandas()
pandas_df

,Agency_Code,Agency_Name,Agency_Type,City,State,Year,Month,Crime_Type,Crime_Solved,Victim_Sex,Victim_Age,Victim_Race,Victim_Ethnicity,Perpetrator_Sex,Perpetrator_Age,Perpetrator_Race,Perpetrator_Ethnicity,Relationship,Weapon,Record_Source
0,OK07205,Tulsa,Municipal Police,Tulsa,Oklahoma,2014,June,Murder or Manslaughter,No,Male,46,Black,Not Hispanic,Unknown,,Unknown,Unknown,Unknown,Handgun,FBI


In [52]:
p_age= sqlCtx.sql("SELECT Perpetrator_Age FROM homicide WHERE Perpetrator_Age<>' '")
p_age.describe().show()

+-------+-----------------+
|summary|  Perpetrator_Age|
+-------+-----------------+
|  count|           638453|
|   mean|20.32269720715542|
| stddev|17.88684181199942|
|    min|                0|
|    max|               99|
+-------+-----------------+



In [53]:
# Missing values
p_out=sqlCtx.sql("SELECT Perpetrator_Age, Crime_Solved, count(*) as count FROM homicide WHERE Perpetrator_Age<10 GROUP BY Perpetrator_Age, Crime_Solved ORDER BY Perpetrator_Age")
p_out.show()

+---------------+------------+------+
|Perpetrator_Age|Crime_Solved| count|
+---------------+------------+------+
|              0|         Yes| 26700|
|              0|          No|189627|
|              1|         Yes|    16|
|              1|          No|    18|
|              2|         Yes|     6|
|              3|         Yes|    24|
|              4|         Yes|    29|
|              5|          No|     2|
|              5|         Yes|    31|
|              6|         Yes|    50|
|              7|         Yes|    48|
|              8|         Yes|    62|
|              9|         Yes|    86|
+---------------+------------+------+



In [54]:
p_age= sqlCtx.sql("SELECT Perpetrator_Age FROM homicide WHERE Perpetrator_Age>10")
p_age.describe().show()

+-------+------------------+
|summary|   Perpetrator_Age|
+-------+------------------+
|  count|            421609|
|   mean|30.766259733544587|
| stddev| 12.77846417945243|
|    min|                11|
|    max|                99|
+-------+------------------+



## Relationship

In [55]:
rel= sqlCtx.sql("SELECT Relationship FROM homicide ORDER BY Relationship")

In [56]:
# Syntactic errors
rel.distinct().show()

+--------------------+
|        Relationship|
+--------------------+
|        Acquaintance|
|           Boyfriend|
|Boyfriend/Girlfriend|
|             Brother|
|  Common-Law Husband|
|     Common-Law Wife|
|            Daughter|
|            Employee|
|            Employer|
|          Ex-Husband|
|             Ex-Wife|
|              Family|
|              Father|
|              Friend|
|          Girlfriend|
|             Husband|
|              In-Law|
|              Mother|
|            Neighbor|
|              Sister|
+--------------------+
only showing top 20 rows



In [57]:
# Count
rel=sqlCtx.sql("SELECT Relationship, count(*) as count FROM homicide GROUP BY Relationship ORDER BY count(*) DESC")
rel.show(28)

+--------------------+------+
|        Relationship| count|
+--------------------+------+
|             Unknown|273013|
|        Acquaintance|126018|
|            Stranger| 96593|
|                Wife| 23187|
|              Friend| 21945|
|          Girlfriend| 16465|
|                 Son|  9904|
|              Family|  9535|
|             Husband|  8803|
|            Daughter|  7539|
|           Boyfriend|  7302|
|            Neighbor|  6294|
|             Brother|  5514|
|              Father|  4361|
|              Mother|  4248|
|              In-Law|  3637|
|     Common-Law Wife|  2477|
|             Ex-Wife|  1973|
|  Common-Law Husband|  1954|
|Boyfriend/Girlfriend|  1383|
|          Stepfather|  1360|
|              Sister|  1292|
|             Stepson|  1170|
|        Stepdaughter|   754|
|          Ex-Husband|   629|
|            Employer|   509|
|            Employee|   384|
|          Stepmother|   211|
+--------------------+------+



In [58]:
# Missing: Relationship Unknown and Crime Solved=Yes
rel=sqlCtx.sql("SELECT Relationship, Crime_Solved, count(*) as count FROM homicide GROUP BY Relationship, Crime_Solved ORDER BY Relationship DESC")
rel.show(55)

+--------------------+------------+------+
|        Relationship|Crime_Solved| count|
+--------------------+------------+------+
|                Wife|         Yes| 23142|
|                Wife|          No|    45|
|             Unknown|         Yes| 94892|
|             Unknown|          No|178121|
|            Stranger|          No|  9231|
|            Stranger|         Yes| 87362|
|             Stepson|         Yes|  1169|
|             Stepson|          No|     1|
|          Stepmother|         Yes|   211|
|          Stepfather|         Yes|  1358|
|          Stepfather|          No|     2|
|        Stepdaughter|         Yes|   754|
|                 Son|          No|    62|
|                 Son|         Yes|  9842|
|              Sister|         Yes|  1288|
|              Sister|          No|     4|
|            Neighbor|         Yes|  6258|
|            Neighbor|          No|    36|
|              Mother|          No|    10|
|              Mother|         Yes|  4238|
|          

In [59]:
rel= sqlCtx.sql("SELECT Relationship, Victim_Sex, Perpetrator_Sex, COUNT(*) FROM homicide GROUP BY Relationship, Victim_Sex, Perpetrator_Sex ORDER BY Relationship")
rel.show(30)

+--------------------+----------+---------------+--------+
|        Relationship|Victim_Sex|Perpetrator_Sex|count(1)|
+--------------------+----------+---------------+--------+
|        Acquaintance|   Unknown|         Female|       4|
|        Acquaintance|      Male|        Unknown|    1893|
|        Acquaintance|    Female|         Female|    2505|
|        Acquaintance|    Female|        Unknown|     268|
|        Acquaintance|      Male|         Female|    6217|
|        Acquaintance|   Unknown|           Male|      25|
|        Acquaintance|   Unknown|        Unknown|       5|
|        Acquaintance|    Female|           Male|   16121|
|        Acquaintance|      Male|           Male|   98980|
|           Boyfriend|    Female|        Unknown|       2|
|           Boyfriend|    Female|           Male|      60|
|           Boyfriend|    Female|         Female|       9|
|           Boyfriend|      Male|        Unknown|      12|
|           Boyfriend|      Male|           Male|     18

In [60]:
view_targ= sqlCtx.sql("SELECT Relationship, count(*) as Tot FROM homicide WHERE Perpetrator_Sex!='Unknown' GROUP BY Relationship")
view_targ.createOrReplaceTempView("view_targ")
target = sqlCtx.sql("SELECT h.Relationship, Perpetrator_Sex, count(*) as Count, count(*)/v.Tot as PercCount FROM homicide as h, view_targ as v WHERE v.Relationship=h.Relationship AND Perpetrator_Sex!='Unknown' GROUP BY h.Relationship, Perpetrator_Sex, v.Tot ORDER BY h.Relationship")
target.show()

+--------------------+---------------+------+--------------------+
|        Relationship|Perpetrator_Sex| Count|           PercCount|
+--------------------+---------------+------+--------------------+
|        Acquaintance|           Male|115126|  0.9295449407357168|
|        Acquaintance|         Female|  8726| 0.07045505926428318|
|           Boyfriend|           Male|   245| 0.03361690450054885|
|           Boyfriend|         Female|  7043|  0.9663830954994511|
|Boyfriend/Girlfriend|           Male|  1085|  0.8469945355191257|
|Boyfriend/Girlfriend|         Female|   196| 0.15300546448087432|
|             Brother|           Male|  5016|  0.9126637554585153|
|             Brother|         Female|   480| 0.08733624454148471|
|  Common-Law Husband|         Female|  1891|  0.9712378017462763|
|  Common-Law Husband|           Male|    56|0.028762198253723677|
|     Common-Law Wife|           Male|  2451|  0.9923076923076923|
|     Common-Law Wife|         Female|    19|0.007692307692307

## Weapon

In [61]:
weap= sqlCtx.sql("SELECT Weapon FROM homicide ORDER BY Weapon")

In [62]:
weap.distinct().show()

+-------------+
|       Weapon|
+-------------+
| Blunt Object|
|     Drowning|
|        Drugs|
|   Explosives|
|         Fall|
|         Fire|
|      Firearm|
|          Gun|
|      Handgun|
|        Knife|
|       Poison|
|        Rifle|
|      Shotgun|
|Strangulation|
|  Suffocation|
|      Unknown|
+-------------+



In [63]:
view_targ= sqlCtx.sql("SELECT Weapon, count(*) as Tot FROM homicide GROUP BY Weapon")
view_targ.createOrReplaceTempView("view_targ")
target = sqlCtx.sql("SELECT h.Weapon, Crime_Solved, count(*) as Count, count(*)/v.Tot as PercCount FROM homicide as h, view_targ as v WHERE v.Weapon=h.Weapon GROUP BY h.Weapon, Crime_Solved, v.Tot ORDER BY h.Weapon")
target.show()

+------------+------------+------+-------------------+
|      Weapon|Crime_Solved| Count|          PercCount|
+------------+------------+------+-------------------+
|Blunt Object|         Yes| 51504| 0.7648692397938726|
|Blunt Object|          No| 15833| 0.2351307602061274|
|    Drowning|          No|   205|0.17026578073089702|
|    Drowning|         Yes|   999|  0.829734219269103|
|       Drugs|         Yes|  1396| 0.8790931989924433|
|       Drugs|          No|   192|0.12090680100755667|
|  Explosives|          No|   149| 0.2774674115456238|
|  Explosives|         Yes|   388| 0.7225325884543762|
|        Fall|          No|    24|0.12631578947368421|
|        Fall|         Yes|   166| 0.8736842105263158|
|        Fire|          No|  2034| 0.3294994330147416|
|        Fire|         Yes|  4139| 0.6705005669852584|
|     Firearm|          No| 22578| 0.4805874840357599|
|     Firearm|         Yes| 24402| 0.5194125159642401|
|         Gun|         Yes|  1324|  0.600181323662738|
|         

## Record Source

In [64]:
rec= sqlCtx.sql("SELECT Record_Source FROM homicide ORDER BY Record_Source")
rec.distinct().show()

+-------------+
|Record_Source|
+-------------+
|          FBI|
|         FOIA|
+-------------+



In [65]:
rec=sqlCtx.sql("SELECT Record_Source, count(*) as count FROM homicide GROUP BY Record_Source ORDER BY count(*) DESC")
rec.show()

+-------------+------+
|Record_Source| count|
+-------------+------+
|          FBI|616647|
|         FOIA| 21807|
+-------------+------+



## Missing Values

In [66]:
for col in df.columns:
    mv_q= sqlCtx.sql("SELECT "+col+", count(*) as CountMissing FROM homicide WHERE "+col+" IS NULL GROUP BY "+ col)
    mv_q.show()

+-----------+------------+
|Agency_Code|CountMissing|
+-----------+------------+
+-----------+------------+

+-----------+------------+
|Agency_Name|CountMissing|
+-----------+------------+
+-----------+------------+

+-----------+------------+
|Agency_Type|CountMissing|
+-----------+------------+
+-----------+------------+

+----+------------+
|City|CountMissing|
+----+------------+
+----+------------+

+-----+------------+
|State|CountMissing|
+-----+------------+
+-----+------------+

+----+------------+
|Year|CountMissing|
+----+------------+
+----+------------+

+-----+------------+
|Month|CountMissing|
+-----+------------+
+-----+------------+

+----------+------------+
|Crime_Type|CountMissing|
+----------+------------+
+----------+------------+

+------------+------------+
|Crime_Solved|CountMissing|
+------------+------------+
+------------+------------+

+----------+------------+
|Victim_Sex|CountMissing|
+----------+------------+
+----------+------------+

+----------+------

In [67]:
# Unknown values
for col in df.columns:
    # Missing: Unknown of solved crime
    mv=sqlCtx.sql("SELECT "+col+", Crime_Solved, count(*) as count FROM homicide WHERE "+col+"=='Unknown' GROUP BY "+col+", Crime_Solved ORDER BY "+col)
    mv.show()   


+-----------+------------+-----+
|Agency_Code|Crime_Solved|count|
+-----------+------------+-----+
+-----------+------------+-----+

+-----------+------------+-----+
|Agency_Name|Crime_Solved|count|
+-----------+------------+-----+
|    Unknown|         Yes|   33|
|    Unknown|          No|   14|
+-----------+------------+-----+

+-----------+------------+-----+
|Agency_Type|Crime_Solved|count|
+-----------+------------+-----+
+-----------+------------+-----+

+----+------------+-----+
|City|Crime_Solved|count|
+----+------------+-----+
+----+------------+-----+

+-----+------------+-----+
|State|Crime_Solved|count|
+-----+------------+-----+
+-----+------------+-----+

+----+------------+-----+
|Year|Crime_Solved|count|
+----+------------+-----+
+----+------------+-----+

+-----+------------+-----+
|Month|Crime_Solved|count|
+-----+------------+-----+
+-----+------------+-----+

+----------+------------+-----+
|Crime_Type|Crime_Solved|count|
+----------+------------+-----+
+----------